In [3]:
import numpy as np

In [4]:
X_train, y_train, X_test, y_test, X_dev, y_dev = np.load('./all_data.npy', allow_pickle = True)

In [5]:
def make_X(x):
    res = []
    x = x[767:]
    for i in range(0, x.shape[0], 768):
#         print(i)
        res.append(np.array(x[i:i+768]))
#     print(np.array(res).shape)
    return np.array(res)

In [6]:
X_train, X_test, X_dev = make_X(X_train), make_X(X_test), make_X(X_dev)

In [7]:
num_classes = 7
timestep = 20

In [8]:
def expand_on_time(X, timestep = 20):
    tmp = np.reshape(X[0], (1, len(X[0])))
#     print(tmp.shape)
    tmp = np.repeat(tmp, timestep, axis = 0)
#     print(tmp.shape)
    ab = np.reshape(tmp, (1, len(tmp[:,:]), len(tmp[0])))
#     print(ab.shape)
#     print(X.shape)
    X = np.repeat(ab, X.shape[0], axis = 0)
#     print(X.shape)
    return X

In [9]:
X_train, X_test, X_dev = expand_on_time(X_train), expand_on_time(X_test), expand_on_time(X_dev)

In [10]:
X_train.shape, X_test.shape, X_dev.shape

((9989, 20, 768), (2610, 20, 768), (1109, 20, 768))

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM
from keras.optimizers import RMSprop, SGD

In [12]:
timestep = 20

model = Sequential()
model.add(LSTM(64, input_shape=(timestep, 768), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Activation('softmax'))
model.add(Dense(num_classes))

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
#rms = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20, 64)            213248    
_________________________________________________________________
dropout (Dropout)            (None, 20, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
activation (Activation)      (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 7)                 231       
Total params: 225,895
Trainable params: 225,895
Non-trainable params: 0
__________________________________________________

In [14]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10)


In [ ]:
hist = model.fit(X_train, y_train, 
                 validation_data = (X_dev, y_dev), 
                 verbose=1, 
                 epochs=10, batch_size=32, 
                 shuffle=False, callbacks=[early_stopping])